### Introduction
This dataset used for this project was sourced from: https://www.kaggle.com/datasets/xiaomengsun/car-insurance-claim-data

The data includes around 8000 records of car insurance policyholders. This includes features such as their age, income, gender, claim hisory and more.

The data include 2 target variables: If a claim has occured, and the total cost of the claim given a claim

In [29]:
data_definitions = {
    'INDEX': 'Unique identifier',
    'TARGET_FLAG': 'Claim has been made',
    'TARGET_AMT': 'Claim value',
    'AGE': 'Age of driver',
    'BLUEBOOK': 'Value of vehicle',
    'CAR_AGE': 'Car age in years',
    'CAR_TYPE': 'Type of vehicle',
    'CAR_USE': 'Usage type',
    'CLM_FRQ': 'Number of claims in past 5 years',
    'EDUCATION': 'Highest level of education completed by policyholder',
    'HOMEKIDS': 'Number of child at policyholder’s address',
    'HOME_VAL': 'Value of policyholder’s home',
    'INCOME': 'Income of policyholder',
    'JOB': 'Type of job held by policyholder',
    'KIDSDRIV': 'Number of children with access to policyholder’s vehicle',
    'MSTATUS': 'Marital status',
    'MVR_PTS': 'Number of points of policyholder’s licence',
    'OLDCLAIM': 'Total value of claims in past 5 years',
    'PARENT1': 'Single parent',
    'RED_CAR': 'Vehicle is red in colour',
    'REVOKED': 'Licence has been revoked in past 7 years',
    'SEX': 'Gender',
    'TIF': 'Policy tenure',
    'TRAVTIME': 'Commute distance',
    'URVANCITY': 'Address area type (urban/rural)',
    'TOJ': "Years policyholder's job has been held for"
}
k = list(data_definitions.keys())
v = list(data_definitions.values())
pd.DataFrame({'Variable': k, 'DESCR': v})

,Variable,DESCR
0,INDEX,Unique identifier
1,TARGET_FLAG,Claim has been made
2,TARGET_AMT,Claim value
3,AGE,Age of driver
4,BLUEBOOK,Value of vehicle
5,CAR_AGE,Car age in years
6,CAR_TYPE,Type of vehicle
7,CAR_USE,Usage type
8,CLM_FRQ,Number of claims in past 5 years
9,EDUCATION,Highest level of education completed by policy...


### Objective
The problem of the project will have 2 parts 
- classification, predicting if a claim has occured, and 
- regression, predicting the value of a claim assuming a claim has been made.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, make_scorer, confusion_matrix, mean_squared_error, mean_absolute_error

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier

from scipy.stats import randint, uniform

,Variable,DESCR
0,INDEX,Unique identifier
1,TARGET_FLAG,Claim has been made
2,TARGET_AMT,Claim value
3,AGE,Age of driver
4,BLUEBOOK,Value of vehicle
5,CAR_AGE,Car age in years
6,CAR_TYPE,Type of vehicle
7,CAR_USE,Usage type
8,CLM_FRQ,Number of claims in past 5 years
9,EDUCATION,Highest level of education completed by policy...


In [6]:
raw_data = pd.read_csv('car_insurance_claim.csv')
pd.set_option('display.max_columns', None)
raw_data.sample(5)

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
3161,237419513,0,20AUG53,46.0,0,15.0,"$36,026",No,"$190,921",Yes,z_F,Bachelors,Professional,26,Private,"$9,700",10,z_SUV,no,$0,0,No,1,$0,13.0,0,Highly Urban/ Urban
6005,974827347,0,24MAY47,52.0,0,13.0,"$49,010",No,"$212,621",Yes,M,Bachelors,z_Blue Collar,13,Commercial,"$14,670",13,Minivan,yes,$0,0,No,0,$0,11.0,0,Highly Urban/ Urban
1388,327882186,1,06MAR62,37.0,2,0.0,$0,No,"$80,883",Yes,z_F,Bachelors,Home Maker,26,Private,"$7,070",1,Sports Car,no,$982,3,No,1,"$2,753",6.0,1,Highly Urban/ Urban
7755,350909425,0,21OCT64,35.0,2,12.0,"$30,111",Yes,$0,z_No,M,z_High School,Clerical,48,Private,"$4,510",6,Pickup,yes,"$7,654",2,No,9,"$2,976",1.0,1,Highly Urban/ Urban
1643,685008787,0,27JUN59,40.0,0,11.0,"$41,477",No,"$185,913",z_No,z_F,PhD,Home Maker,16,Private,"$16,710",4,Minivan,no,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban


In [32]:
# Create copy of Data
data = raw_data.copy()
data.sample(5)

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
6393,47083764,0,01DEC55,43.0,0,8.0,"$166,192",No,$0,z_No,M,Masters,NaN,52,Commercial,"$21,110",1,Van,yes,"$4,962",3,No,2,$0,NaN,0,Highly Urban/ Urban
8285,311741524,0,27JUL56,43.0,0,11.0,"$30,183",No,"$87,809",Yes,z_F,Masters,Home Maker,17,Private,"$8,820",6,z_SUV,no,$0,0,No,0,$0,NaN,0,Highly Urban/ Urban
9201,536604822,0,23JUL50,49.0,0,NaN,"$94,283",No,"$270,384",Yes,M,Masters,NaN,35,Commercial,"$29,550",1,Panel Truck,yes,$0,0,No,1,$0,19.0,0,z_Highly Rural/ Rural
639,116202466,0,18NOV52,46.0,0,13.0,NaN,No,"$140,734",Yes,M,z_High School,z_Blue Collar,50,Commercial,"$11,700",1,Pickup,yes,$0,0,No,0,$0,NaN,0,z_Highly Rural/ Rural
9487,899945663,0,31JAN55,44.0,0,11.0,"$42,275",No,"$190,963",Yes,M,z_High School,z_Blue Collar,34,Commercial,"$11,330",6,Pickup,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban


In [34]:
# Change Col names to something more interpretable
col_names = {
    'KIDSDRIV': 'num_young_drivers',
    'BIRTH': 'date_of_birth',
    'AGE': 'age',
    'HOMEKIDS': 'num_of_children',
    'YOJ': 'years_job_held_for',
    'INCOME': 'income',
    'PARENT1': 'single_parent',
    'HOME_VAL': 'value_of_home',
    'MSTATUS': 'married',
    'GENDER': 'gender',
    'EDUCATION': 'highest_education',
    'OCCUPATION': 'occupation',
    'TRAVTIME': 'commute_dist',
    'CAR_USE': 'type_of_use',
    'BLUEBOOK': 'vehicle_value',
    'TIF': 'policy_tenure',
    'CAR_TYPE': 'vehicle_type',
    'RED_CAR': 'is_red_vehicle',
    'OLDCLAIM': '5_year_total_claims_value',
    'CLM_FREQ': '5_year_num_of_claims',
    'REVOKED': 'licence_revoked',
    'MVR_PTS': 'license_points',
    'CLM_AMT': 'new_claim_value',
    'CAR_AGE': 'vehicle_age',
    'CLAIM_FLAG': 'is_claim',
    'URBANICITY': 'area_type'
}

data.rename(columns=col_names, inplace=True)
data.head()


,ID,num_young_drivers,date_of_birth,age,num_of_children,years_job_held_for,income,single_parent,value_of_home,married,gender,highest_education,occupation,commute_dist,type_of_use,vehicle_value,policy_tenure,vehicle_type,red_vehicle,5_year_total_claims_value,5_year_num_of_claims,licence_revoked,license_points,new_claim_value,vehicle_age,is_claim,address_type
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,M,Bachelors,Manager,26,Private,"$21,970",1,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,z_F,z_High School,Clerical,5,Private,"$4,010",4,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,M,<High School,z_Blue Collar,32,Private,"$15,440",7,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


In [36]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10302 entries, 0 to 10301
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          10302 non-null  int64  
 1   KIDSDRIV    10302 non-null  int64  
 2   BIRTH       10302 non-null  object 
 3   AGE         10295 non-null  float64
 4   HOMEKIDS    10302 non-null  int64  
 5   YOJ         9754 non-null   float64
 6   INCOME      9732 non-null   object 
 7   PARENT1     10302 non-null  object 
 8   HOME_VAL    9727 non-null   object 
 9   MSTATUS     10302 non-null  object 
 10  GENDER      10302 non-null  object 
 11  EDUCATION   10302 non-null  object 
 12  OCCUPATION  9637 non-null   object 
 13  TRAVTIME    10302 non-null  int64  
 14  CAR_USE     10302 non-null  object 
 15  BLUEBOOK    10302 non-null  object 
 16  TIF         10302 non-null  int64  
 17  CAR_TYPE    10302 non-null  object 
 18  RED_CAR     10302 non-null  object 
 19  OLDCLAIM    10302 non-nul

In [42]:
data.isna().sum().sort_values(ascending=False)

occupation                   665
vehicle_age                  639
value_of_home                575
income                       570
years_job_held_for           548
age                            7
ID                             0
policy_tenure                  0
is_claim                       0
new_claim_value                0
license_points                 0
licence_revoked                0
5_year_num_of_claims           0
5_year_total_claims_value      0
red_vehicle                    0
vehicle_type                   0
commute_dist                   0
vehicle_value                  0
type_of_use                    0
num_young_drivers              0
highest_education              0
gender                         0
married                        0
single_parent                  0
num_of_children                0
date_of_birth                  0
address_type                   0
dtype: int64

In [48]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)

In [50]:
# Currency Based Columns
currency_cols = ['income', 'value_of_home', 'vehicle_value', '5_year_total_claims_value', 'new_claim_value']

# Replace $ and ',' from currency column values
def format_currency_col(data, cols):
    for col in cols:
        data[col] = data[col]
    

,income,value_of_home,vehicle_value,5_year_total_claims_value,new_claim_value
0,"$67,349",$0,"$14,230","$4,461",$0
1,"$91,449","$257,252","$14,940",$0,$0
2,"$52,881",$0,"$21,970",$0,$0
3,"$16,039","$124,191","$4,010","$38,690",$0
4,NaN,"$306,251","$15,440",$0,$0
...,...,...,...,...,...
10297,"$164,669","$386,273","$13,270",$0,$0
10298,"$107,204","$332,591","$24,490",$0,$0
10299,"$39,837","$170,611","$13,820",$0,$0
10300,"$43,445","$149,248","$22,550",$0,$0
